<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/finetuning/embeddings/finetune_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune Embeddings

In this notebook, we show users how to finetune their own embedding models.

We go through three main sections:
1. Preparing the data (our `generate_qa_embedding_pairs` function makes this easy)
2. Finetuning the model (using our `SentenceTransformersFinetuneEngine`)
3. Evaluating the model on a validation knowledge corpus

## Generate Corpus

First, we create the corpus of text chunks by leveraging LlamaIndex to load some financial PDFs, and parsing/chunking into plain text chunks.

In [1]:
%pip install datasets
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-finetuning
%pip install llama-index-readers-file
%pip install llama-index-embeddings-huggingface
%pip install "transformers[torch]"
%pip install llama-index-llms-gemini

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/808.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/808.7 kB ? eta -:--:--
   ---------------------------------------- 808.7/808.7 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/647.0 kB ? eta -:--:--
   ---------------------------------------- 647.0/647.0 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.0
    Uninstalling openai-1.61.0:
      Successfully uninstalled openai-1.61.0



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



...


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.16
    Uninstalling google-ai-generativelanguage-0.6.16:
      Successfully uninstalled google-ai-generativelanguage-0.6.16
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.0.11 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import json

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode

Download Data

In [3]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

In [5]:
import json
from llama_index.finetuning import EmbeddingQAFinetuneDataset
# Или ваш точный путь импорта, если он другой:
# from llama_index.core.llama_dataset.legacy.embedding import EmbeddingQAFinetuneDataset

# 1. Определяем наш собственный класс-наследник
class Utf8EmbeddingQAFinetuneDataset(EmbeddingQAFinetuneDataset):
    """
    Наследник EmbeddingQAFinetuneDataset, который читает JSON
    с явным указанием кодировки UTF-8.
    """
    @classmethod
    def from_json(cls, path: str) -> "Utf8EmbeddingQAFinetuneDataset":
        """Загружает датасет из JSON файла с кодировкой UTF-8."""
        try:
            # Используем 'r' для чтения и encoding='utf-8'
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            # Возвращаем экземпляр НАШЕГО класса
            return cls(**data)
        except FileNotFoundError:
            print(f"Ошибка: Файл не найден по пути {path}")
            raise # Пробрасываем исключение дальше
        except json.JSONDecodeError as e:
            print(f"Ошибка декодирования JSON в файле {path}: {e}")
            raise
        except Exception as e:
            print(f"Неожиданная ошибка при чтении файла {path}: {e}")
            raise

# 2. Используем НАШ класс для загрузки данных
train_file_path = "train_dataset_structured_5q_by_channel_with_prefixes.json"
val_file_path = "validation_dataset_structured_5q_by_channel.json"

try:
    print(f"Загрузка тренировочного датасета: {train_file_path}")
    train_dataset = Utf8EmbeddingQAFinetuneDataset.from_json(train_file_path)
    print(f"Тренировочный датасет '{train_file_path}' успешно загружен.")

    print(f"Загрузка валидационного датасета: {val_file_path}")
    val_dataset = Utf8EmbeddingQAFinetuneDataset.from_json(val_file_path)
    print(f"Валидационный датасет '{val_file_path}' успешно загружен.")

    print("Датасеты готовы к использованию.")
    # Ваш дальнейший код...

except Exception as e:
     # Ошибки уже будут напечатаны внутри from_json
     print(f"Не удалось загрузить один или оба датасета. Прерывание.")


Загрузка тренировочного датасета: train_dataset_structured_5q_by_channel_with_prefixes.json
Тренировочный датасет 'train_dataset_structured_5q_by_channel_with_prefixes.json' успешно загружен.
Загрузка валидационного датасета: validation_dataset_structured_5q_by_channel.json
Валидационный датасет 'validation_dataset_structured_5q_by_channel.json' успешно загружен.
Датасеты готовы к использованию.


## Run Embedding Finetuning

In [6]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

In [7]:
finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id="intfloat/multilingual-e5-small",
    model_output_path="test_model",
    val_dataset=val_dataset,
)

C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


INFO:datasets:PyTorch version 2.5.1 available.
PyTorch version 2.5.1 available.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-small
Load pretrained SentenceTransformer: intfloat/multilingual-e5-small


C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\hf_cache\hub\models--intfloat--multilingual-e5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\alexa\AppData\Local\Packages\P

In [9]:
!pip install --upgrade llama-index-finetuning

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\alexa\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
# --- Ячейка Jupyter Notebook ---

# 1. Импорты
import os
import json
from typing import Any, Optional
from llama_index.finetuning import SentenceTransformersFinetuneEngine
from llama_index.finetuning.embeddings.common import EmbeddingQAFinetuneDataset
from llama_index.core.base.embeddings.base import BaseEmbedding
from llama_index.core.embeddings.utils import resolve_embed_model

print("Импорты завершены.")

# 2. Определение исправленного класса-наследника
class PatchedSentenceTransformersFinetuneEngine(SentenceTransformersFinetuneEngine):
    """
    Наследник SentenceTransformersFinetuneEngine для исправления несовместимости
    аргумента 'resume_from_checkpoint' с новыми версиями sentence-transformers.
    """
    def __init__(self, *args, **kwargs):
        """Инициализация с безопасной обработкой resume_from_checkpoint."""
        # Сохраняем и удаляем аргумент перед вызовом родительского конструктора
        self._init_resume_from_checkpoint = kwargs.pop('resume_from_checkpoint', False)
        print(f"Patched __init__: resume_from_checkpoint received as {self._init_resume_from_checkpoint}")
        super().__init__(*args, **kwargs)
        # Восстанавливаем значение атрибута, если он нужен для логики checkpoint_path
        # (хотя в методе fit мы его не используем)
        self.resume_from_checkpoint = self._init_resume_from_checkpoint
        print(f"Patched __init__: Set self.resume_from_checkpoint to {self.resume_from_checkpoint}")


    def finetune(self, **train_kwargs: Any) -> None:
        """Переопределенный метод finetune без аргумента resume_from_checkpoint."""
        print("--- Using patched finetune method (without resume_from_checkpoint) ---")
        # Формируем словарь аргументов для метода fit базовой модели sentence-transformers
        fit_kwargs = {
            'train_objectives': [(self.loader, self.loss)],
            'epochs': self.epochs,
            'warmup_steps': self.warmup_steps,
            'output_path': self.model_output_path,
            'show_progress_bar': self.show_progress_bar,
            'evaluator': self.evaluator,
            'evaluation_steps': self.evaluation_steps,
            'checkpoint_path': self.checkpoint_path,
            'checkpoint_save_steps': self.checkpoint_save_steps,
            'checkpoint_save_total_limit': self.checkpoint_save_total_limit,
            # 'resume_from_checkpoint' НЕ ПЕРЕДАЕТСЯ
        }
        # Добавляем любые дополнительные аргументы, переданные в finetune
        fit_kwargs.update(train_kwargs)

        print(f"Calling self.model.fit with args: {list(fit_kwargs.keys())}")
        # Вызываем метод fit базовой модели sentence_transformers
        self.model.fit(**fit_kwargs)
        print("--- Patched finetune method finished ---")

print("Класс PatchedSentenceTransformersFinetuneEngine определен.")

# 3. Класс для загрузки данных с UTF-8 (из предыдущего шага)
class Utf8EmbeddingQAFinetuneDataset(EmbeddingQAFinetuneDataset):
    @classmethod
    def from_json(cls, path: str) -> "Utf8EmbeddingQAFinetuneDataset":
        try:
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            return cls(**data)
        except FileNotFoundError:
            print(f"Ошибка: Файл не найден по пути {path}")
            raise
        except json.JSONDecodeError as e:
            print(f"Ошибка декодирования JSON в файле {path}: {e}")
            raise
        except Exception as e:
            print(f"Неожиданная ошибка при чтении файла {path}: {e}")
            raise

print("Класс Utf8EmbeddingQAFinetuneDataset определен.")

# 4. Загрузка датасетов с использованием исправленного класса
train_file_path = "train_dataset_structured_5q_by_channel_with_prefixes.json"
val_file_path = "validation_dataset_structured_5q_by_channel.json"
train_dataset = None
val_dataset = None

try:
    print(f"Загрузка тренировочного датасета: {train_file_path}")
    train_dataset = Utf8EmbeddingQAFinetuneDataset.from_json(train_file_path)
    print(f"Тренировочный датасет '{train_file_path}' успешно загружен ({len(train_dataset.queries)} запросов).")

    print(f"Загрузка валидационного датасета: {val_file_path}")
    val_dataset = Utf8EmbeddingQAFinetuneDataset.from_json(val_file_path)
    print(f"Валидационный датасет '{val_file_path}' успешно загружен ({len(val_dataset.queries)} запросов).")

except Exception as e:
     print(f"Не удалось загрузить датасеты. Ошибка: {e}")

# 5. Инициализация и запуск файнтюнинга с ИСПРАВЛЕННЫМ классом
if train_dataset and val_dataset:
    print("\nИнициализация finetune engine с Patched классом...")
    try:
        finetune_engine = PatchedSentenceTransformersFinetuneEngine(
            dataset=train_dataset,  # Передаем загруженный train_dataset
            model_id="intfloat/multilingual-e5-small", # Модель, которую файнтюним
            model_output_path="test_model_patched",    # Куда сохранить результат
            val_dataset=val_dataset, # Передаем загруженный val_dataset
            epochs=1,               # Пример количества эпох (настройте по необходимости)
            batch_size=16,          # Пример размера батча (настройте по необходимости)
            evaluation_steps=100,   # Пример шага оценки
            # resume_from_checkpoint=False, # Этот аргумент убран из __init__ и finetune
            save_checkpoints=False, # Отключаем чекпоинты для простоты, если они не нужны
            show_progress_bar=True
        )
        print("Finetune engine инициализирован. Запуск finetune...")

        # Запускаем файнтюнинг
        finetune_engine.finetune()

        print("\nФайнтюнинг завершен!")

        # Можно получить модель после файнтюнинга
        # finetuned_model = finetune_engine.get_finetuned_model()
        # print("Доступ к модели после файнтюнинга получен.")

    except Exception as e:
        print(f"\nПроизошла ошибка во время инициализации или файнтюнинга: {e}")
        import traceback
        traceback.print_exc()

else:
     print("\nФайнтюнинг не запущен, так как датасеты не были загружены.")


Импорты завершены.
Класс PatchedSentenceTransformersFinetuneEngine определен.
Класс Utf8EmbeddingQAFinetuneDataset определен.
Загрузка тренировочного датасета: train_dataset_structured_5q_by_channel_with_prefixes.json
Тренировочный датасет 'train_dataset_structured_5q_by_channel_with_prefixes.json' успешно загружен (270175 запросов).
Загрузка валидационного датасета: validation_dataset_structured_5q_by_channel.json
Валидационный датасет 'validation_dataset_structured_5q_by_channel.json' успешно загружен (35475 запросов).

Инициализация finetune engine с Patched классом...
Patched __init__: resume_from_checkpoint received as False
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-small
Load pretrained SentenceTransformer: intfloat/multilingual-e5-small
Patched __init__: Set self.resume_from_checkpoint to False
Finetune engine

C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\trainer.py:205: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SentenceTransformerTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Traceback (most recent call last):
  File "C:\Users\alexa\AppData\Local\Temp\ipykernel_35644\1694651730.py", line 116, in <module>
    finetune_engine.finetune()
  File "C:\Users\alexa\AppData\Local\Temp\ipykernel_35644\1694651730.py", line 54, in finetune
    self.model.fit(**fit_kwargs)
  File "C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\fit_mixin.py", line 356, in fit
    trainer = SentenceTransformerTrainer(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\alexa\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

In [ ]:
embed_model = finetune_engine.get_finetuned_model()

In [ ]:
embed_model

HuggingFaceEmbedding(model_name='test_model', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x2cc3d5cd0>, tokenizer_name='test_model', max_length=512, pooling=<Pooling.CLS: 'cls'>, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

## Evaluate Finetuned Model

In this section, we evaluate 3 different embedding models:
1. proprietary OpenAI embedding,
2. open source `BAAI/bge-small-en`, and
3. our finetuned embedding model.

We consider 2 evaluation approaches:
1. a simple custom **hit rate** metric
2. using `InformationRetrievalEvaluator` from sentence_transformers

We show that finetuning on synthetic (LLM-generated) dataset significantly improve upon an opensource embedding model.

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

### Define eval function

**Option 1**: We use a simple **hit rate** metric for evaluation:
* for each (query, relevant_doc) pair,
* we retrieve top-k documents with the query,  and
* it's a **hit** if the results contain the relevant_doc.

This approach is very simple and intuitive, and we can apply it to both the proprietary OpenAI embedding as well as our open source and fine-tuned embedding models.

In [ ]:
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(
        nodes, embed_model=embed_model, show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

**Option 2**: We use the `InformationRetrievalEvaluator` from sentence_transformers.

This provides a more comprehensive suite of metrics, but we can only run it against the sentencetransformers compatible models (open source and our finetuned model, *not* the OpenAI embedding model).

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer
from pathlib import Path


def evaluate_st(
    dataset,
    model_id,
    name,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    evaluator = InformationRetrievalEvaluator(
        queries, corpus, relevant_docs, name=name
    )
    model = SentenceTransformer(model_id)
    output_path = "results/"
    Path(output_path).mkdir(exist_ok=True, parents=True)
    return evaluator(model, output_path=output_path)

### Run Evals

#### OpenAI

Note: this might take a few minutes to run since we have to embed the corpus and queries

In [ ]:
ada = OpenAIEmbedding()
ada_val_results = evaluate(val_dataset, ada)

In [ ]:
df_ada = pd.DataFrame(ada_val_results)

In [ ]:
hit_rate_ada = df_ada["is_hit"].mean()
hit_rate_ada

0.8779904306220095

### BAAI/bge-small-en

In [ ]:
bge = "local:BAAI/bge-small-en"
bge_val_results = evaluate(val_dataset, bge)

Generating embeddings:   0%|          | 0/418 [00:00<?, ?it/s]

  0%|          | 0/836 [00:00<?, ?it/s]

In [ ]:
df_bge = pd.DataFrame(bge_val_results)

In [ ]:
hit_rate_bge = df_bge["is_hit"].mean()
hit_rate_bge

0.7930622009569378

In [ ]:
evaluate_st(val_dataset, "BAAI/bge-small-en", name="bge")

FileNotFoundError: [Errno 2] No such file or directory: 'results/Information-Retrieval_evaluation_bge_results.csv'

### Finetuned

In [ ]:
finetuned = "local:test_model"
val_results_finetuned = evaluate(val_dataset, finetuned)

In [ ]:
df_finetuned = pd.DataFrame(val_results_finetuned)

In [ ]:
hit_rate_finetuned = df_finetuned["is_hit"].mean()
hit_rate_finetuned

In [ ]:
evaluate_st(val_dataset, "test_model", name="finetuned")

### Summary of Results

#### Hit rate

In [ ]:
df_ada["model"] = "ada"
df_bge["model"] = "bge"
df_finetuned["model"] = "fine_tuned"

We can see that fine-tuning our small open-source embedding model drastically improve its retrieval quality (even approaching the quality of the proprietary OpenAI embedding)!

In [ ]:
df_all = pd.concat([df_ada, df_bge, df_finetuned])
df_all.groupby("model").mean("is_hit")

#### InformationRetrievalEvaluator

In [ ]:
df_st_bge = pd.read_csv(
    "results/Information-Retrieval_evaluation_bge_results.csv"
)
df_st_finetuned = pd.read_csv(
    "results/Information-Retrieval_evaluation_finetuned_results.csv"
)

We can see that embedding finetuning improves metrics consistently across the suite of eval metrics

In [ ]:
df_st_bge["model"] = "bge"
df_st_finetuned["model"] = "fine_tuned"
df_st_all = pd.concat([df_st_bge, df_st_finetuned])
df_st_all = df_st_all.set_index("model")
df_st_all